# take video and split it into scenes

In [5]:
import cv2
import os
import ffmpeg
import numpy as np
import subprocess

def save_scene(video_path, output_folder, start_time, end_time, scene_number):
    output_file = os.path.join(output_folder, f"scene_{scene_number}.avi")
    ffmpeg.input(video_path, ss=start_time/1000, to=end_time/1000).output(output_file, vcodec='huffyuv').run()

# TODO: simplify by using ffmpeg filter scene, also goes in error at the last scene, irrelevant for now but maybe I can fix it
def split_video_into_scenes(video_path, output_folder, threshold=100):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    prev_frame = None
    scene_start = 0
    scene_number = 1

    # parse the video frame by frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if prev_frame is not None:
            # if frame is different enough from previous, consider it a scene change
            diff = cv2.absdiff(prev_frame, frame)
            mean_diff = diff.mean()
            if mean_diff > threshold:
                # get times of scene start and end, save new video between those times
                scene_end = cap.get(cv2.CAP_PROP_POS_MSEC)
                save_scene(video_path, output_folder, scene_start, scene_end, scene_number)
                scene_start = scene_end
                scene_number += 1

        prev_frame = frame.copy()

    # Save the last scene
    save_scene(video_path, output_folder, scene_start, cap.get(cv2.CAP_PROP_POS_MSEC), scene_number)

    cap.release()

input_path = "Tears_of_Steel_1080p.mov"
output_path = "Tears_of_Steel_1080p"
os.makedirs(output_path, exist_ok=True)
# watch the video, identify scene changes, set threshold accordingly 
# split_video_into_scenes(input_path, output_path, threshold=45)

In [6]:
# take scene and split it into frames

scene_path = 'Tears_of_Steel_1080p/scene_5.avi'

def split_video_into_frames(video_path):
    video_name, _ = video_path.rsplit('.', 1)
    # if folder exists, empty it, otherwise create it
    if os.path.exists(video_name):
        for file in os.listdir(video_name):
            os.remove(video_name + '/' + file)
    else:
        os.mkdir(video_name)
    subprocess.run(f'ffmpeg -i {scene_path} {video_name}/frame%04d.png', shell=True)

# split_video_into_frames(scene_path)

In [7]:
# take each frame and 
## 1. split into blocks
### 1.1. whiten certain blocks
### 1.2. reduce the number of blocks
## 2. recreate frame
## 3. save reconstructed frames in scene folder

def split_image_into_squares(image, l):
    """
    Split an image into squares of a specific size.

    Args:
    - image: numpy array representing the image with shape [n, m, c]
    - l: integer representing the side length of each square

    Returns:
    - numpy array with shape [n//l, m//l, l, l, c] containing the squares
    """
    n, m, c = image.shape
    num_rows = n // l
    num_cols = m // l
    result = np.zeros((num_rows, num_cols, l, l, c), dtype=image.dtype)
    for i in range(num_rows):
        for j in range(num_cols):
            result[i, j] = image[i*l:(i+1)*l, j*l:(j+1)*l, :]
    return result

def whiten_squares_by_indices(squares, filter_factor):
    """
    Whitens squares based on their indices.

    Args:
    - squares: numpy array with shape [n, m, l, l, c] containing the squares

    Returns:
    - numpy array with shape [n, m, l, l, c] containing the squares with selected ones whitened
    """
    n, m, l, _, c = squares.shape
    whitened_squares = squares.copy()
    for i in range(n):
        for j in range(m):
            if i % filter_factor != 0 or j % filter_factor != 0:
                whitened_squares[i, j] = np.ones((l, l, c), dtype=squares.dtype) * 255  # Make the square white
    return whitened_squares

def filter_squares_by_indices(squares, filter_factor):
    """
    Filter squares based on their indices.

    Args:
    - squares: numpy array with shape [n, m, l, l, c] containing the squares

    Returns:
    - numpy array with shape [n_filtered, m_filtered, l, l, c] containing the filtered squares
    """
    filtered_squares = squares[::filter_factor, ::filter_factor]
    return filtered_squares

def reconstruct_image(squares):
    """
    Reconstruct the original image from split squares.

    Args:
    - squares: numpy array with shape [n, m, l, l, c] containing the split squares

    Returns:
    - numpy array representing the reconstructed image
    """
    n, m, l, _, c = squares.shape
    num_rows = n * l
    num_cols = m * l
    image = np.zeros((num_rows, num_cols, c), dtype=squares.dtype)
    for i in range(n):
        for j in range(m):
            image[i*l:(i+1)*l, j*l:(j+1)*l, :] = squares[i, j]
    return image

def process_frame(image, square_size, filter_factor):
    """
    Process an image by splitting, filtering, and reconstructing it.

    Args:
    - image: numpy array representing the image with shape [n, m, c]
    - square_size: integer representing the side length of each square

    Returns:
    - numpy array representing the processed image
    """
    squares = split_image_into_squares(image, square_size)
    filtered_squares = filter_squares_by_indices(squares, filter_factor)
    whitened_squares = whiten_squares_by_indices(squares, filter_factor)
    reconstructed_filtered = reconstruct_image(filtered_squares)
    reconstructed_whitened = reconstruct_image(whitened_squares)
    return reconstructed_filtered, reconstructed_whitened

def save_processed_frames(reconstructed_filtered, reconstructed_whitened, frame_path):

    folder, frame = frame_path.rsplit('/', 1)

    # Create the filtered directory if it doesn't exist
    filtered_folder = folder + '/reconstructed_filtered'
    os.makedirs(filtered_folder, exist_ok=True)
    # save filtered frame
    cv2.imwrite(filtered_folder + '/' + frame, reconstructed_filtered, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

    # Create the whitened directory if it doesn't exist
    whitened_folder = folder + '/reconstructed_whitened'
    os.makedirs(whitened_folder, exist_ok=True)
    # save whitened frame
    cv2.imwrite(whitened_folder + '/' + frame, reconstructed_whitened, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

scene_name, _ = scene_path.rsplit('.', 1)
# for frame_path in os.listdir(scene_name):
#     frame = cv2.imread(scene_name + '/' + frame_path)
#     reconstructed_filtered, reconstructed_whitened = process_frame(frame, 16, 2)
#     save_processed_frames(reconstructed_filtered, reconstructed_whitened, scene_name + '/' + frame_path)

In [8]:
# take all frames and recreate video

def get_frame_rate(video_file):
    """
    Get the frame rate of a video file using OpenCV.

    Args:
    - video_file: Path to the video file.

    Returns:
    - Frame rate of the video file.
    """
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print("Error: Couldn't open the video file.")
        return None

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return frame_rate

def reconstruct_video_from_frames(input_folder, frame_rate):
    """
    Create a lossless video from frames in a folder using OpenCV with FFV1 codec.

    Args:
    - input_folder: Path to the folder containing the frames.
    - output_video_path: Path to save the output video.
    - frame_rate: Frame rate of the output video.

    Returns:
    - None
    """
    frame_files = sorted(os.listdir(input_folder))
    output_video_path = input_folder + '.avi'

    # Get the first frame to obtain its dimensions
    first_frame_path = os.path.join(input_folder, frame_files[0])
    first_frame = cv2.imread(first_frame_path)
    frame_height, frame_width, _ = first_frame.shape

    # Initialize VideoWriter object with FFV1 codec for lossless compression
    fourcc = cv2.VideoWriter_fourcc(*'FFV1')  # FFV1 codec for lossless compression
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height))

    # Iterate through each frame file and write it to the video
    for frame_file in frame_files:
        frame_path = os.path.join(input_folder, frame_file)
        frame = cv2.imread(frame_path)
        out.write(frame)

    # Release the VideoWriter object
    out.release()

frame_rate = get_frame_rate(scene_path)
# # TODO: overwrite if exists
reconstruct_video_from_frames(scene_name + '/reconstructed_filtered', frame_rate)
reconstruct_video_from_frames(scene_name + '/reconstructed_whitened', frame_rate)


# encode video and check size reduction from encoded original


In [15]:
def encode_video(input_video_path, output_video_path, codec, bitrate=None, crf=None):
    """
    Encode a video file with the specified codec and bitrate.

    Args:
    - input_video_path: Path to the input video file.
    - output_video_path: Path to save the output video.
    - codec: Codec to use for encoding (default: libx264).
    - bitrate: Bitrate for the output video (default: '10M' for 10 Mbps).

    Returns:
    - None
    """
    if bitrate:
        command = ['ffmpeg', '-y', '-i', input_video_path, '-c:v', codec, '-b:v', bitrate, output_video_path]
    elif crf:
        command = ['ffmpeg', '-y', '-i', input_video_path, '-c:v', codec, '-crf', str(crf), output_video_path]
    else:
        return 'Please specify either a target bitrate or a compression rate factor'
    subprocess.run(command)

# constants
codec = 'libx264'
bitrate = '1M'
crf = 23

# # encode original
original = scene_path
encoded_original = scene_name + '/encoded_original.avi'
encode_video(original, encoded_original, codec, crf=20)

# # encode filtered
filtered = scene_name + '/reconstructed_filtered.avi'
encoded_filtered = scene_name + '/encoded_filtered.avi'
encode_video(filtered, encoded_filtered, codec, crf=20)

# # encode whitened
whitened = scene_name + '/reconstructed_whitened.avi'
encoded_whitened = scene_name + '/encoded_whitened.avi'
encode_video(whitened, encoded_whitened, codec, crf=20)

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-5)
  configuration: --prefix=/home/itec/emanuele/.conda/envs/inpainting --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1706918361713/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1706918361713/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1706918361713/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1706918361713/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-vaapi --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --

TypeError: posix.wait() takes no arguments (1 given)

In [16]:
# compare sizes
encoded_original_size = os.stat(encoded_original).st_size
encoded_filtered_size = os.stat(encoded_filtered).st_size
encoded_whitened_size = os.stat(encoded_whitened).st_size

print(f'encoded original: {encoded_original_size}')
print(f'encoded filtered size reduction: {1 - encoded_filtered_size/encoded_original_size}%')
print(f'encoded whitened size reduction: {1 - encoded_whitened_size/encoded_original_size}%')

# inpaint video and check similarity with encoded original



encoded original: 8322386
encoded filtered size reduction: 0.5996179461034372%
encoded whitened size reduction: 0.31713285108381173%
